# 🛠 Install Libraries

In [ ]:
# bbox-utility, check https://github.com/awsaf49/bbox for source code
!pip install -q /kaggle/input/loguru-lib-ds/loguru-0.5.3-py3-none-any.whl
!pip install -q /kaggle/input/bbox-lib-ds

# 📚 Import Libraries

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import glob
import shutil
import sys
sys.path.append('../input/tensorflow-great-barrier-reef')
import torch
from PIL import Image

# Configuration

In [ ]:
ROOT_DIR  = '/kaggle/input/tensorflow-great-barrier-reef/'
CKPT_PATH = '/kaggle/input/1st-train/yolov5s6_trial5_img2000_epoch25_batch8.pt'
IMG_SIZE  = 4000
CONF      = 0.28
IOU       = 0.30
AUGMENT   = True

In [ ]:
train = pd.read_csv('/kaggle/input/tensorflow-great-barrier-reef/train.csv')
train['image_path'] = f'{ROOT_DIR}/train_images/video_'+train.video_id.astype(str)+'/'+train.video_frame.astype(str)+'.jpg'
train['annotations'] = train['annotations'].apply(eval)
train.head()

## Number of BBoxes

In [ ]:
train['num_bbox'] = train['annotations'].progress_apply(lambda x: len(x))
data = (train.num_bbox>0).value_counts()/len(train)*100
print(f"No BBox: {data[0]:0.2f}% | With BBox: {data[1]:0.2f}%")

# 🔨 Helper

In [ ]:
# check https://github.com/awsaf49/bbox for source code of following utility functions
from bbox.utils import coco2yolo, coco2voc, voc2yolo, voc2coco
from bbox.utils import draw_bboxes, load_image
from bbox.utils import clip_bbox, str2annot, annot2str

def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

def get_imgsize(row):
    row['width'], row['height'] = imagesize.get(row['image_path'])
    return row

np.random.seed(32)
colors = [(np.random.randint(255), np.random.randint(255), np.random.randint(255))\
          for idx in range(1)]

In [ ]:
def load_model(ckpt_path, conf=0.25, iou=0.50):
    model = torch.hub.load('/kaggle/input/yolov5-lib-ds',
                           'custom',
                           path=ckpt_path,
                           source='local',
                           force_reload=True)  # local repo
    model.conf = conf  # NMS confidence threshold
    model.iou  = iou  # NMS IoU threshold
    model.classes = None   # (optional list) filter by class, i.e. = [0, 15, 16] for persons, cats and dogs
    model.multi_label = False  # NMS multiple labels per box
    model.max_det = 1000  # maximum number of detections per image
    return model

In [ ]:
def predict(model, img, size=768, augment=False):
    height, width = img.shape[:2]
    results = model(img, size=size, augment=augment)  # custom inference size
    preds   = results.pandas().xyxy[0]
    bboxes  = preds[['xmin','ymin','xmax','ymax']].values
    if len(bboxes):
        bboxes  = voc2coco(bboxes,height,width).astype(int)
        confs   = preds.confidence.values
        return bboxes, confs
    else:
        return [],[]
    
def format_prediction(bboxes, confs):
    annot = ''
    if len(bboxes)>0:
        for idx in range(len(bboxes)):
            xmin, ymin, w, h = bboxes[idx]
            conf             = confs[idx]
            annot += f'{conf} {xmin} {ymin} {w} {h}'
            annot +=' '
        annot = annot.strip(' ')
    return annot

def show_img(img, bboxes, bbox_format='yolo'):
    names  = ['starfish']*len(bboxes)
    labels = [0]*len(bboxes)
    img    = draw_bboxes(img = img,
                           bboxes = bboxes, 
                           classes = names,
                           class_ids = labels,
                           class_name = True, 
                           colors = colors, 
                           bbox_format = bbox_format,
                           line_thickness = 2)
    return Image.fromarray(img).resize((600, 300))

In [ ]:
!mkdir -p /root/.config/Ultralytics
!cp /kaggle/input/yolov5-font/Arial.ttf /root/.config/Ultralytics/

# 🔭 Inference

## Run Inference on **Train**

In [ ]:
model = load_model(CKPT_PATH, conf=CONF, iou=IOU)
tmp_df = train[train.num_bbox>1].sample(20)
image_path = tmp_df.image_path.tolist()
sample_anno = tmp_df.annotations.tolist()
for idx in range(5):
    img = cv2.imread(image_path[idx])[...,::-1]
    sample_bbox = [[i['x'],i['y'],i['width'],i['height']] for i in sample_anno[idx]]
    bboxes, confis = predict(model, img, size=IMG_SIZE, augment=AUGMENT)
    print('Ground Truth')
    display(show_img(img, sample_bbox, bbox_format='coco'))
    print('Prediction')
    display(show_img(img, bboxes, bbox_format='coco'))

## Run Inference on **Test**

In [ ]:
import greatbarrierreef
env = greatbarrierreef.make_env()# initialize the environment
iter_test = env.iter_test()      # an iterator which loops over the test set and sample submission

In [ ]:
model = load_model(CKPT_PATH, conf=CONF, iou=IOU)
for idx, (img, pred_df) in enumerate(tqdm(iter_test)):
    bboxes, confs  = predict(model, img, size=IMG_SIZE, augment=AUGMENT)
    annot          = format_prediction(bboxes, confs)
    pred_df['annotations'] = annot
    env.predict(pred_df)
    if idx<3:
        display(show_img(img, bboxes, bbox_format='coco'))

# 👀 Check Submission

In [ ]:
sub_df = pd.read_csv('submission.csv')
sub_df.head()